tgb - 10/16/2019 - Automatizing the calculation of the potential decomposition for COSMO because it takes too long and the connection resets before it's done

In [1]:
import util.Landau_Potential_Diffusion as Landau
import util.curvature as curve
import util.pdf as PDF

import xarray as xr
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import datetime
from skimage import measure
from scipy import ndimage
import scipy.integrate as sin
from scipy.optimize import curve_fit
import matplotlib as mpl

def bin_mid_to_edge(binm):
    bine = 0.5*(binm[:-1]+binm[1:]) # bin_edges[1:-1]
    return np.concatenate(([bine[0]-(bine[2]-bine[1])],bine,[bine[-1]+(bine[2]-bine[1])]))

path = '/project/s916/davidle/RCE-MIP/simulations/RCE_300_3km_506x506/output/'

RCE300 = xr.open_mfdataset(path+'lfff????????.nc')

dt = 3600; # Timestep in seconds
tcoor = dt*np.arange(0,RCE300.time.shape[0])

t_range = np.arange(0,np.size(RCE300.time)-2)
dMSE_dt = (RCE300.FMSE.values[t_range+2,:,:]-RCE300.FMSE.values[t_range,:,:])/(2*dt)
dMSE_dt = np.concatenate((dMSE_dt[0:1,:,:],dMSE_dt,np.tile(dMSE_dt[-1,:,:],(1,1,1))),axis=0)

SEF = -RCE300.SHFL_S - RCE300.LHFL_S
SW = RCE300.ASOB_T-RCE300.ASOB_S
LW = RCE300.ATHB_T-RCE300.ATHB_S
DIAB = LW+SW+SEF
ADV_MSE = dMSE_dt-DIAB

bin0=np.percentile(a=RCE300.FMSE[-24*7:,:,:],q=50,axis=(0,1,2))

tmp,binm,Vtot = Landau.Landau_energy(RCE300.FMSE,dMSE_dt,bin0,N_bins=30)
tmp,tmp,Vsef = Landau.Landau_energy(RCE300.FMSE,SEF,bin0,N_bins=bin_mid_to_edge(binm))
tmp,tmp,Vsw = Landau.Landau_energy(RCE300.FMSE,SW,bin0,N_bins=bin_mid_to_edge(binm))
tmp,tmp,Vlw = Landau.Landau_energy(RCE300.FMSE,LW,bin0,N_bins=bin_mid_to_edge(binm))
tmp,tmp,Vadv = Landau.Landau_energy(RCE300.FMSE,ADV_MSE,bin0,N_bins=bin_mid_to_edge(binm))

X_pot = (binm-bin0)/2.5e6
# Save potentials in .pkl file since they take a long time to calculate
hf = open('PKL_DATA/10_15_VCOSMO.pkl','wb')
V_data = {"X_pot":X_pot,"Vtot":Vtot,"Vsef":Vsef,"Vsw":Vsw,"Vlw":Vlw,"Vadv":Vadv,"binm":binm,"bin0":bin0}
pickle.dump(V_data,hf)
hf.close()

/users/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: In xarray version 0.13 `auto_combine` will be deprecated.
  coords=coords)
/users/tbeucler/miniconda3/lib/python3.7/site-packages/xarray/backends/api.py:783: FutureWarning: The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset` to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order the datasets are supplied in in future, please use the
new `combine_nested` function (or the `combine='nested'` option to
open_mfdataset).
  coords=coords)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/users/tbeucler/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-94ba0d88ac01>", line 39, in <module>
    tmp,binm,Vtot = Landau.Landau_energy(RCE300.FMSE,dMSE_dt,bin0,N_bins=30)
  File "/users/tbeucler/2019_WMI/dev/tbeucler/util/Landau_Potential_Diffusion.py", line 145, in Landau_energy
    else: binmid,V,tmp,tmp = potential(order_p,order_p_tendency,N_bins)
  File "/users/tbeucler/2019_WMI/dev/tbeucler/util/Landau_Potential_Diffusion.py", line 51, in potential
    forcing)
  File "/users/tbeucler/2019_WMI/dev/tbeucler/util/Landau_Potential_Diffusion.py", line 37, in conditional_avg_and_std
    Ym[ibin-1],Ystd[ibin-1] = weighted_avg_and_std(field_y,w)
  File "/users/tbeucler/2019_WMI/dev/tbeucler/util/Landau_Potential_Diffusion.py", line 22, in weighted_avg_and_std
    variance = np.average((values-average)**2, weights

KeyboardInterrupt: 